In [ ]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [ ]:
players <- read_csv("players.csv")

players

In [ ]:
sessions <- read_csv("sessions.csv")

sessions

In [ ]:
new_players <- players |>
select(experience, subscribe, played_hours, gender, Age)
new_players

In [ ]:
new_players$experience[new_players$experience == "Pro"] <- 4

new_players$experience[new_players$experience == "Veteran"] <- 5

new_players$experience[new_players$experience == "Amateur"] <- 1

new_players$experience[new_players$experience == "Regular"] <- 3

new_players$experience[new_players$experience == "Beginner"] <- 2

In [ ]:
newest_players <- new_players |>
mutate(gender = as_factor(gender), subscribe = as_factor(subscribe),
      experience = as.integer(as_factor(experience)))

newest_players

In [ ]:
experience_played_hours_plot <- newest_players |>
   ggplot(aes(x = experience, y = played_hours, color = subscribe)) + 
   geom_point() + 
   xlab("Experience (standardized)") +
   ylab("played_hours (standardized)") +
   ggtitle("played_hours vs. experience") +
   labs(color = "Class of subscribe")

experience_played_hours_plot

In [ ]:
gender_Age_plot <- newest_players |>
   ggplot(aes(x = gender, y = Age, color = subscribe)) + 
   geom_point() + 
   xlab("gender (standardized)") +
   ylab("Age (standardized)") +
   ggtitle("Age vs. gender") +
   labs(color = "Class of subscribe")

gender_Age_plot

In [ ]:
played_hours_Age_plot <- newest_players |>
   ggplot(aes(x = played_hours, y = Age, color = subscribe)) + 
   geom_point() + 
   xlab("played_hours (standardized)") +
   ylab("Age (standardized)") +
   ggtitle("Age vs. hours") +
   labs(color = "Class of subscribe")

played_hours_Age_plot

In [ ]:
experience_Age_plot <- newest_players |>
   ggplot(aes(x = experience, y = Age, color = subscribe)) + 
   geom_point() + 
   xlab("played_hours (standardized)") +
   ylab("Age (standardized)") +
   ggtitle("Age vs. experience") +
   labs(color = "Class of subscribe")

experience_Age_plot

In [ ]:
minecraft_split <- initial_split(newest_players, prop = 0.75, strata = subscribe)  
minecraft_train <- training(minecraft_split)   
minecraft_test <- testing(minecraft_split)

In [ ]:
minecraft_recipe <- recipe(subscribe ~ experience + Age +  played_hours, data = newest_players) |>
   step_scale(experience, Age, played_hours) |>
   step_center(experience, Age, played_hours)

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
      set_engine("kknn") |>
      set_mode("classification")

minecraft_fit <- workflow() |>
      add_recipe(minecraft_recipe) |>
      add_model(knn_spec) |>
      fit(data = newest_players)

minecraft_fit

In [ ]:
minecraft_test_predictions <- predict(minecraft_fit , minecraft_test) |>
      bind_cols(minecraft_test)